In [47]:
%matplotlib inline

# COS 424 Project: Matrix completion and use of analogies
In this project, we tackle the matrix completion problem from a cognitive viewpoint, at Marr's computational level of analysis. 

In [48]:
import os, sys, gc
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import itertools

## Prior distributions
We have two prior distributions: the distribution over class-level mappings, $p({\bf m}|{\bf \theta})$, and the prior over domain usage, $p(({\bf \eta}, {\bf \theta})|\gamma)$.

# Mapping probability
We have that ${\bf x}|{\bf \theta} \, \sim \, \text{Multinomial({\bf \theta} )}$, where ${\bf x}$ is the count vector for ${\bf m}$ over classes in ${\bf \theta}$.

The PDF of a Multinomial is as follows:

$p(x_1, ..., x_k., ..., x_K | {\bf \theta}) \, = \, \frac{\Gamma(\sum_k x_k + 1)}{\prod_k \Gamma(x_k + 1)}\prod_{k=1}^{K}{\bf \theta}_k^{x_k}$

In [49]:
def gamma_functionRB(integer):
    """Implementation of the integeric gamma function"""
    return np.math.factorial(integer - 1)

def power_functionRB(probability, count):
    return probability ** count



In [50]:
def analogy_prior(numSystems):
    """Currently just uniform"""
    return 1 / numSystems

In [51]:
def mapping_likelihood(mapVec, probVec):
    """mapping vector assigns each of n elements to m classes. 
    probability vector has underlying frequency probability of the m classes.
    Should return the probability of mapping vector under a multinomial
    parameterized by these probabilities.
    
    
    N.B. As we are interested in sequences, we do not need normalizing constant!
    https://stats.stackexchange.com/questions/66447/understanding-multinomial-distribution"""
    outputClasses = np.shape(probVec)[0]
    print('Number of output classes: ', outputClasses)
    print('checking mapping vec: {0}'.format(mapVec))
    countVec = np.bincount(mapVec, minlength=outputClasses)
    print('countVec: ', countVec)
    numerator = gamma_functionRB(np.sum(countVec) + 1)
    print('numerator: ', numerator)
    denominator = np.prod([gamma_functionRB(x + 1) for x in countVec])
    print('denominator:', denominator)
    probability = np.prod([power_functionRB(x[0], x[1]) for x in zip(probVec, countVec)])
    print('probability: ', probability)
    # see definition
    #mappingProb = (numerator / denominator) * probability
    mappingProb = probability
    print('mapping probability is: {0}'.format(mappingProb))
    return mappingProb

In [52]:
testProbsMan = np.array([0.4, 0.5, 0.1])
testCountsMan = np.array([2, 3, 1])
numerator = 6 * 5 * 4 * 3 * 2
denominator = (2) * (3 * 2)
probs = (0.4**2) * (0.5 ** 3) * (0.1)
prob = (numerator / denominator) * probs
print(prob)
testMapVecMan = np.concatenate([np.repeat(x[0], x[1]) for x in zip(np.arange(3), testCountsMan)])
print(testMapVecMan)

0.12000000000000002
[0 0 1 1 1 2]


In [53]:
print(mapping_likelihood(testMapVecMan, testProbsMan))
rv = ss.multinomial(6, testProbsMan)
print(rv.pmf(testCountsMan))

Number of output classes:  3
checking mapping vec: [0 0 1 1 1 2]
countVec:  [2 3 1]
numerator:  720
denominator: 12
probability:  0.0020000000000000005
mapping probability is: 0.0020000000000000005
0.0020000000000000005
0.1200000000000001


In [54]:
testClasses = 5
testDraws = 20
testProbs = np.random.dirichlet(np.ones(testClasses))
print(testProbs.shape)
countVec = np.random.multinomial(testDraws, testProbs)
mapVec = np.concatenate([np.repeat(x[0], x[1]) for x in zip(np.arange(testClasses), countVec)])
countVec2 = np.bincount(mapVec, minlength=testClasses)
print(mapVec, '\n', countVec, '\n', testProbs, np.sum(testProbs))
print(countVec2)
cs = np.cumsum(countVec2)
rv = ss.multinomial(testDraws, testProbs)
mulScipy = rv.pmf(countVec2)
mulRB = mapping_likelihood(mapVec, testProbs)
print(mulScipy == mulRB, mulScipy, mulRB)

(5,)
[0 0 2 2 2 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4] 
 [ 2  0  3  5 10] 
 [0.23112096 0.00736464 0.11198134 0.15729107 0.49224199] 1.0
[ 2  0  3  5 10]
Number of output classes:  5
checking mapping vec: [0 0 2 2 2 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4]
countVec:  [ 2  0  3  5 10]
numerator:  2432902008176640000
denominator: 5225472000
probability:  6.031461593719195e-12
mapping probability is: 6.031461593719195e-12
False 0.0028081587698871435 6.031461593719195e-12


In [55]:

def extract_class_info(partitionVector):
    """Basic module for extracting class information from 
    partition vector"""
    numEntities = np.shape(partitionVector)[0]
    classes = np.unique(partitionVector)
    numClasses = np.shape(classes)[0]
    classIdxs = [np.where(partitionVector==x)[0] for x in classes]
    classSizes = [np.shape(x)[0] for x in classIdxs]
    classInfo = {'iden' : classes, 'num': numClasses, 'idxs': classIdxs,
            'ents': numEntities, 'sizes': classSizes}
    
    return classInfo

#zTest = np.array([0, 0, 1, 1])
#classInfo = extract_class_info(zTest)


# In[3]:


def reduce_to_class_interactions(relationMatrix, partitionVector, 
                                 classInfo, paramMat):
    """This function takes a matrix of inter-entity observed interactions 
    and a vector assigning entities to classes,
    and returns two matrices of interclass counts and noncounts"""
    
    # returns sorted (ascending) unique classes
     
        
    m = np.zeros(paramMat.shape)
    mNeg = m.copy()
        
    # goes through all pairs of class assignments and counts them
    for i, a in enumerate(classInfo['iden']):
        aIdx = classInfo['idxs'][i]
        
        # select out class A rows
        classRelMat = relationMatrix[aIdx, :]
        for j, b in enumerate(classInfo['iden']):
            bIdx = classInfo['idxs'][j]
            
            # subselect out class B columns
            tempRelMat = classRelMat[:, bIdx]
            
            # count number of interactive entities in two classes
            total = tempRelMat.shape[0] * tempRelMat.shape[1]
            counts = np.sum(tempRelMat)
            m[a, b] = counts
            mNeg[a, b] = total - counts
            
    return m, mNeg

def apply_mask(targetRelationMatrix, unobservedIdxs, unobservedIdxsMapped, m, mNeg):
    """Expects idxs to be a 2xnu matrix, where nu is number of unobserved relations, and idxs are
    entities. Returns a masked (m and) mNeg"""
    print('m pre: {0}'.format(m))
    print('mNeg pre: {0}'.format(mNeg))
    trueValue = targetRelationMatrix[unobservedIdxs[0], unobservedIdxs[1]]
    print('True value: {0}'.format(trueValue))
    if trueValue == 0:
        mNeg[unobservedIdxsMapped[0], unobservedIdxsMapped[1]] -= 1
    elif trueValue == 1:
        m[unobservedIdxsMapped[0], unobservedIdxsMapped[1]] -= 1
    print('m post: {0}'.format(m))
    print('mNeg post: {0}'.format(mNeg))
    return m, mNeg

def probability_fnRB(paramMat, mMasked, mNegMasked):
    """Calculates the unnormalized binomial probability of each matrix entry at class level. Expects all
    matrices to be same size (classes by classes)"""
    
    powerMatM = np.power(paramMat, mMasked)
    powerMatMNeg = np.power(1 - paramMat, mNegMasked)
    print('power matrices: \n {0} \n {1}'.format(powerMatM, powerMatMNeg))
    prod1 = np.prod(powerMatM)
    prod1Neg = np.prod(powerMatMNeg)

    finalProd = prod1 * prod1Neg
    print('power matrices reduced: \n {0} \n {1}'.format(prod1, prod1Neg))
    print('final prod: \n {0}'.format(finalProd))
    return finalProd

In [56]:
def relation_likelihood(targetRelationMat, targetUnobservedIdx, targetUnobservedIdxMapped, paramMat, freqVec, mappingVec):
    classInfo = extract_class_info(mappingVec)
    m, mNeg = reduce_to_class_interactions(targetRelationMat, mappingVec, classInfo, paramMat)
    mMasked, mNegMasked = apply_mask(targetRelationMat, targetUnobservedIdx, targetUnobservedIdxMapped, m, mNeg)
    relationProb = probability_fnRB(paramMat, mMasked, mNegMasked)
    print('relation probability is: {0}'.format(relationProb))
    return relationProb

In [57]:
def predict_value(paramMat, targetUnobservedIdxMapped):
    """Fill in single missing value"""
    return paramMat[targetUnobservedIdxMapped[0], targetUnobservedIdxMapped[1]]

## Generate mapping vectors
As we are marginalizing over mapping vectors, we need some way to generate all possible mappings. This is equivalent to an N balls (new relations) into M bins (old classes) problem; or alternatively taking the Cartesian product of M possible allocations for each ball. Clearly this is exponential, with $M^N$ combinations.

In [58]:
N = 3
M = 3

def cartesian_product(N, M):
    return np.array(list(itertools.product(range(M), repeat=N)))
#print(cartesian_product(N, M))

# Pipeline for a single system in memory
Given a single system in memory, generate mappings. For each mappings, generate prediction, and analogical weight. Sum over weighted predictions.

In [64]:
def analogical_weight(targetRelationMat, targetUnobservedIdx, targetUnobservedIdxMapped, paramMat, freqVec, mappingVec, numSystems = 1):
    return relation_likelihood(targetRelationMat, targetUnobservedIdx, targetUnobservedIdxMapped, paramMat, freqVec, mappingVec) * mapping_likelihood(mappingVec, freqVec) * analogy_prior(numSystems)

def weighted_prediction(targetRelationMat, targetUnobservedIdx, targetUnobservedIdxMapped, paramMat, freqVec, mappingVec):
    pred = predict_value(paramMat, targetUnobservedIdxMapped)
    print('pred is: {0}'.format(pred))
    weight = analogical_weight(targetRelationMat, targetUnobservedIdx, targetUnobservedIdxMapped, paramMat, freqVec, mappingVec, numSystems = 1)
    print('weight is: {0}'.format(weight))
    return  pred, weight

In [86]:
def generate_mappings(targetRelationMat, paramMat):
    return cartesian_product(targetRelationMat.shape[0], paramMat.shape[0])

def final_prediction(targetRelationMat, targetUnobservedIdx, paramMat, freqVec):
    runningPrediction = 0.0
    weights = []
    mappings = []
    for mappingVec in generate_mappings(targetRelationMat, paramMat):
        print('\n mapping vec: {0}'.format(mappingVec))
        print('target unobserved idx: {0}'.format(targetUnobservedIdx))
        targetUnobservedIdxMapped = np.array([mappingVec[targetUnobservedIdx[0]], mappingVec[targetUnobservedIdx[1]]])
        print('mapped prediction indices: {0}'.format(targetUnobservedIdxMapped))
        pred, weight = weighted_prediction(targetRelationMat, targetUnobservedIdx, targetUnobservedIdxMapped, paramMat, freqVec, mappingVec)
        mappings.append(mappingVec)
        weights.append(weight)
        runningPrediction += pred
    return runningPrediction, weights, mappings

def return_best_analogy(weightList, mappingVecs, paramMat):
    bestMapping = mappingVecs[np.argmax(weightList)]
    interpretationMat = paramMat[bestMapping, :]
    interpretationMat2 = interpretationMat[:, bestMapping]

    return interpretationMat2
    

In [87]:
R = np.array([[1, 1], [1, 0]])
u = np.array([0, 1])
eta1 = np.array([[0.8, 0.8], [0.8, 0.2]])
eta2 = 1 - eta1
pi = np.array([0.6, 0.4])

pred, weights, mappings = final_prediction(R, u, eta1, pi)
print()
print('Final prediction')
print(pred, np.sum(weights), pred * np.sum(weights))
print(return_best_analogy(weights, mappings, eta1))

pred, weights, mappings = final_prediction(R, u, eta2, pi)
print()
print('Final prediction')
print(pred, np.sum(weights), pred * np.sum(weights))
print(return_best_analogy(weights, mappings, eta2))



 mapping vec: [0 0]
target unobserved idx: [0 1]
mapped prediction indices: [0 0]
pred is: 0.8
m pre: [[3. 0.]
 [0. 0.]]
mNeg pre: [[1. 0.]
 [0. 0.]]
True value: 1
m post: [[2. 0.]
 [0. 0.]]
mNeg post: [[1. 0.]
 [0. 0.]]
power matrices: 
 [[0.64 1.  ]
 [1.   1.  ]] 
 [[0.2 1. ]
 [1.  1. ]]
power matrices reduced: 
 0.6400000000000001 
 0.19999999999999996
final prod: 
 0.128
relation probability is: 0.128
Number of output classes:  2
checking mapping vec: [0 0]
countVec:  [2 0]
numerator:  2
denominator: 2
probability:  0.36
mapping probability is: 0.36
weight is: 0.046079999999999996

 mapping vec: [0 1]
target unobserved idx: [0 1]
mapped prediction indices: [0 1]
pred is: 0.8
m pre: [[1. 1.]
 [1. 0.]]
mNeg pre: [[0. 0.]
 [0. 1.]]
True value: 1
m post: [[1. 0.]
 [1. 0.]]
mNeg post: [[0. 0.]
 [0. 1.]]
power matrices: 
 [[0.8 1. ]
 [0.8 1. ]] 
 [[1.  1. ]
 [1.  0.8]]
power matrices reduced: 
 0.6400000000000001 
 0.8
final prod: 
 0.5120000000000001
relation probability is: 0.51200000

In [89]:
R = np.array([[1, 1, 1], [1, 0, 1], [0, 1, 0]])
u = np.array([0, 1])
eta1 = np.array([[0.8, 0.8, 0.8], [0.8, 0.2, 0.8], [0.2, 0.8, 0.2]])
eta2 = 1 - eta1
pi = np.array([0.4, 0.3, 0.3])

pred, weights, mappings = final_prediction(R, u, eta1, pi)
print()
print('Final prediction')
print(pred, np.sum(weights), pred * np.sum(weights))
print(return_best_analogy(weights, mappings, eta1))

pred, weights, mappings = final_prediction(R, u, eta2, pi)
print()
print('Final prediction')
print(pred, np.sum(weights), pred * np.sum(weights))
print(return_best_analogy(weights, mappings, eta2))



 mapping vec: [0 0 0]
target unobserved idx: [0 1]
mapped prediction indices: [0 0]
pred is: 0.8
m pre: [[6. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
mNeg pre: [[3. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
True value: 1
m post: [[5. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
mNeg post: [[3. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
power matrices: 
 [[0.32768 1.      1.     ]
 [1.      1.      1.     ]
 [1.      1.      1.     ]] 
 [[0.008 1.    1.   ]
 [1.    1.    1.   ]
 [1.    1.    1.   ]]
power matrices reduced: 
 0.3276800000000001 
 0.007999999999999995
final prod: 
 0.002621439999999999
relation probability is: 0.002621439999999999
Number of output classes:  3
checking mapping vec: [0 0 0]
countVec:  [3 0 0]
numerator:  6
denominator: 6
probability:  0.06400000000000002
mapping probability is: 0.06400000000000002
weight is: 0.00016777216

 mapping vec: [0 0 1]
target unobserved idx: [0 1]
mapped prediction indices: [0 0]
pred is: 0.8
m pre: [[3. 2. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
mNeg pre: [[1. 0. 0.]
 [1. 1. 0.]
 [0